In [ ]:
#import os
#os.environ['KERAS_BACKEND']='theano'
%run init.ipy
#import logging    
#logging.getLogger('keras').setLevel(logging.INFO)


from utils import *
from miregularizer import kde_entropy, kde_condentropy

In [ ]:
d=load_mnist(max_train_items=1000)
X_train_Kvar = K.variable(d.X_train)


In [ ]:
from keras.layers import Layer
from keras.regularizers import ActivityRegularizer

class MIRegularizer(ActivityRegularizer):
    def __init__(self, alpha):
        super(MIRegularizer, self).__init__()
        self.alpha = alpha
        
    def get_h(self):
        return kde_entropy(self.layer.input, K.exp(self.layer.logvar)+K.exp(self.layer.kdelayer.logvar))
    
    def get_hcond(self):
        return kde_condentropy(self.layer.input, K.exp(self.layer.kdelayer.logvar))
    
    def get_mi(self):
        return self.get_h() - self.get_hcond()
    
    def __call__(self, loss):
        if not hasattr(self, 'layer'):
            raise Exception('Need to call `set_layer` on ActivityRegularizer instance before calling the instance.')
            
        mi = self.get_mi()
        regularized_loss = loss + self.alpha * mi
        return K.in_train_phase(regularized_loss, loss)

    
class GaussianNoise2(Layer):
    # with variable noise
    def __init__(self, init_logvar, kdelayer, alpha=1.0, *kargs, **kwargs):
        self.supports_masking = True
        self.init_logvar = init_logvar
        self.uses_learning_phase = True
        self.kdelayer = kdelayer
        self.alpha = alpha
        super(GaussianNoise2, self).__init__(*kargs, **kwargs)
        
    def build(self, input_shape):
        super(GaussianNoise2, self).build(input_shape)
        self.logvar = K.variable(float(self.init_logvar))
        # self.trainable_weights = [self.logvar,]
        self.trainable_weights = []

        mireg = MIRegularizer(alpha=self.alpha)
        mireg.layer = self
        self.regularizers.append(mireg)
        
    def call(self, x, mask=None):
        noise_x = x + K.sqrt(K.exp(self.logvar)) * K.random_normal(shape=K.shape(x), mean=0., std=1)
        return K.in_train_phase(noise_x, x)

class KDEParamLayer(Layer):
    # with variable noise
    def __init__(self, init_logvar):
        self.init_logvar = init_logvar
        super(KDEParamLayer, self).__init__()
        
    def build(self, input_shape):
        super(KDEParamLayer, self).__init__()
        self.logvar = K.variable(float(self.init_logvar))
        self.trainable_weights = []

    def call(self, x, mask=None):
        return x
        

def kde_entropy_from_dists_loo(dists, N, dims, var):
    # dists should have large values on diagonal
    dists2 = dists / (2*var)
    normconst = (dims/2.0)*K.log(2*np.pi*var)
    lprobs = logsumexp(-dists2, axis=1) - np.log(N-1) - normconst
    h = -K.mean(lprobs)
    return h

        
        
from keras.layers.normalization import BatchNormalization

HIDDEN_DIM = 5
kdelayer   = KDEParamLayer(init_logvar=-3)
noiselayer = GaussianNoise2(init_logvar=-3, kdelayer=kdelayer, alpha=10.0)
model = Sequential()
#model.add(Dense(2*HIDDEN_DIM, input_dim=d.X_train.shape[1], activation='tanh'))
model.add(Dense(HIDDEN_DIM, input_dim=d.X_train.shape[1], activation='tanh'))
#model.add(BatchNormalization())
#model.add(MILayerTrainable(HIDDEN_DIM, alpha=.1, initlogvar=-1., add_noise=True, entropy_only=False))
model.add(BatchNormalization())
model.add(kdelayer)
model.add(noiselayer)
model.add(Dense(d.nb_classes, activation='softmax')) # , activity_regularizer=MIRegularizer(0, gaussian_var)))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
#from keras.layers import Input

from keras.callbacks import Callback
from scipy.misc import logsumexp
import scipy
import time

class KDETrain(Callback):
    def __init__(self, kdelayer, *kargs, **kwargs):
        super(KDETrain, self).__init__(*kargs, **kwargs)
        self.kdelayer = kdelayer
        #self.kde_train = make_kdevar_update_func(model, kdelayer)
        self.nlayerinput = lambda x: K.function([model.layers[0].input, K.learning_phase()], [noiselayer.input])([x,1])[0]
        Kdists = K.placeholder()
        Klogvar = K.placeholder()
        N, dims = d.X_train.shape
        self.obj = K.function([Kdists, Klogvar, K.learning_phase()], [kde_entropy_from_dists_loo(dists, N, dims, K.exp(Klogvar))])
        self.jac = K.function([Kdists, Klogvar, K.learning_phase()], K.gradients(kde_entropy_from_dists_loo(dists, N, dims, K.exp(Klogvar)), Klogvar))

    def get_optimum_sigma(self):
        vals = self.nlayerinput(d.X_train)
        dists = self.get_dists(vals)
        dists += 10e20 * np.eye(dists.shape[0])
        stime = time.time()
        r= scipy.optimize.minimize(lambda x: self.obj([dists, x, 1])[0], -10, jac=lambda x: self.jac([dists, x, 1])[0])
        return r.x[0]
    
    @staticmethod
    def get_dists(output):
        N, dims = output.shape

        # Kernel density estimation of entropy
        y1 = output[None,:,:]
        y2 = output[:,None,:]

        dists = np.sum((y1-y2)**2, axis=2) 
        return dists

    #    def negloglikelihood(logvar):
    #        return self.kde_entropy_from_dists_loo(dists, vals.shape[0], vals.shape[1], np.exp(logvar))
    #                                                  method='powell')
    #@staticmethod
    #def kde_entropy_from_dists_loo(dists, N, dims, var):
    #    # diagonal entries should be large
    #    dists2 = dists / (2*var)
    #    normconst = (dims/2.0)*np.log(2*np.pi*var)
    #    lprobs = logsumexp(-dists2, axis=1) - np.log(N-1) - normconst
    #    h = -np.mean(lprobs)
    #    return h

    #def on_epoch_end(self, epoch, logs={}):
    #    for _ in range(10):
    #        print self.kde_train([d.X_train,d.Y_train, np.ones(len(d.X_train)), 1])
    #        print 'kdeLV=%.5f' % K.get_value(kdelayer.logvar)
    def on_epoch_begin(self, epoch, logs={}):
        best_val = self.get_optimum_sigma()
        K.set_value(kdelayer.logvar, best_val)
        print 'kdeLV=%.5f' % K.get_value(kdelayer.logvar)
        
        #for _ in range(10):
        #    print self.kde_train([d.X_train,d.Y_train, np.ones(len(d.X_train)), 1])
        #    print 'kdeLV=%.5f' % K.get_value(kdelayer.logvar)
    
class ReportVars(Callback):
    def __init__(self, kdelayer, noiselayer, *kargs, **kwargs):
        super(ReportVars, self).__init__(*kargs, **kwargs)
        self.noiselayer = noiselayer
        self.kdelayer = kdelayer
        self.mifunc = None
        #self.kde_train = make_kdevar_update_func(model, kdelayer)
        #self.sample_weights = np.ones(len(d.X_train))

    #def on_batch_end(self, batch, log={}):
        
    def on_epoch_end(self, epoch, logs={}):
        if self.mifunc is None:
            self.mifunc = K.function([self.model.layers[0].input, K.learning_phase()], 
                                     [noiselayer.regularizers[0].get_mi(), 
                                      noiselayer.regularizers[0].get_h(),
                                      noiselayer.regularizers[0].get_hcond()])
        lv1, lv2 = K.get_value(kdelayer.logvar), K.get_value(noiselayer.logvar)
        logs['kdeLV']   = lv1
        logs['noiseLV'] = lv2
        print 'kdeLV=%.5f, noiseLV=%.5f, mi=%s' % (lv1, lv2, self.mifunc([d.X_train,1]))
   

In [ ]:
batch_size = 100
callbacks = []
callbacks = [KDETrain(kdelayer=kdelayer),ReportVars(noiselayer=noiselayer,kdelayer=kdelayer),]

hist = model.fit(d.X_train, d.Y_train, nb_epoch=50,
                 batch_size=batch_size, validation_split=0.1, verbose=2, 
                 callbacks=callbacks)
#print hist.history

In [ ]:
f1 = K.function([model.layers[0].input], [noiselayer.input])
x1=f1([d.X_train])[0]
print x1.shape

In [ ]:
f1 = K.function([model.layers[0].input], [noiselayer.input])
x1=f1([d.X_train])[0]

from scipy.misc import logsumexp
def kde_entropy_np(output, var):
    N, dims = output.shape
    
    normconst = (dims/2.0)*np.log(2*np.pi*var)
            
    # Kernel density estimation of entropy
    y1 = output[None,:,:]
    y2 = output[:,None,:]
    
    dists = np.sum((y1-y2)**2, axis=2) / (2*var)

    # Removes effect of diagonals, i.e. leave-one-out entropy
    normCount = N-1
    diagvals = 10e20*np.eye(N)
    dists = dists + diagvals
    #print np.exp(-dists)
    lprobs = logsumexp(-dists) - np.log(normCount) - normconst
    
    h = -np.mean(lprobs)
    return h

def kdecondentropy2(output, var):
    dims = int(output.shape[1])
    normconst = (dims/2.0)*np.log(2*np.pi*np.e*var)
    return normconst #+ (dims/2.0)

import sklearn.decomposition
x2 = sklearn.decomposition.PCA(20, whiten=True).fit_transform(x1)

l=[]
logvars = np.linspace(-10, 0, 20)
for logvar in logvars:
    pass
logvar = 5
if True:
    print logvar, kde_entropy_np(x2, np.exp(logvar)), kdecondentropy2(x2, np.exp(logvar))
    l.append( kde_entropy_np(x2, np.exp(logvar)) )

In [ ]:
plt.plot(logvars, l)

In [ ]:
for iterndx in range(25):
    print iterndx, " ",
    #model.fit(X_train, Y_train, nb_epoch=1, batch_size=250, validation_split=0.1) # , verbose=1)
    model.fit(d.X_train, d.Y_train, nb_epoch=1, batch_size=500, validation_split=0.1) # , verbose=1)
    if hasattr(model.layers[-2],'logvar'):
        print K.get_value(model.layers[-3].logvar)
        print K.get_value(model.layers[-2].logvar)
                

In [ ]:
if False:
    from miregularizer import *

    HIDDEN_DIM = 20

    model = Sequential()

    model.add(Dense(2*HIDDEN_DIM, input_dim=d.X_train.shape[1], activation='tanh'))
    model.add(Dense(HIDDEN_DIM, input_dim=d.X_train.shape[1], activation='tanh'))
    if False:
        pass #.97
    elif True:    
        model.add(MILayer(HIDDEN_DIM, alpha=.1, trainablevar=True, initlogvar=-1., add_noise=True, entropy_only=False))
    elif True:    # not clustered enough, no advatnage
        model.add(MILayer(HIDDEN_DIM, alpha=.1, trainablevar=False, initlogvar=-1., add_noise=False, entropy_only=False))
    elif True:    # not clustered enough, no advatnage
        model.add(MILayer(HIDDEN_DIM, alpha=.2, trainablevar=False, initlogvar=0.5, add_noise=False, entropy_only=False))
    elif True:   # .965, but very "clustered" due to small initlogvar
        model.add(MILayer(HIDDEN_DIM, alpha=.2, trainablevar=False, initlogvar=-2, add_noise=False, entropy_only=False))
    elif False:
        model.add(MILayer(HIDDEN_DIM, alpha=.05, trainablevar=True, initlogvar=0.5, add_noise=False, entropy_only=False))
    elif True:  
        model.add(MILayer(HIDDEN_DIM, alpha=.2, trainablevar=False, initlogvar=-0.5, add_noise=False, entropy_only=False))
    elif True:  
        model.add(MILayer(HIDDEN_DIM, alpha=.4, trainablevar=False, initlogvar=-0.5, add_noise=False, entropy_only=False))
    elif True:   # reaches .972 on 2 layer net 2*20, 20  .also with alpha=.2
        model.add(MILayer(HIDDEN_DIM, alpha=.1, trainablevar=False, initlogvar=-0.5, add_noise=False, entropy_only=False))
    elif True:  # reaches .97 on 2 layer net 2*20, 20 .
        model.add(MILayer(HIDDEN_DIM, alpha=.05, trainablevar=False, initlogvar=-0.5, add_noise=False, entropy_only=False))
    elif True:
        model.add(MILayer(HIDDEN_DIM, alpha=.01, trainablevar=True, initlogvar=0.5, add_noise=True, entropy_only=False))
    elif False:
        # mi w. noise, fixed var
        model.add(MILayer(HIDDEN_DIM, alpha=.2, trainablevar=False, initlogvar=0.1, add_noise=True, entropy_only=False))
    elif False:
        # mi w. noise, trainable var
        model.add(MILayer(HIDDEN_DIM, alpha=.2, trainablevar=True, initlogvar=0.1, add_noise=True, entropy_only=False))
    elif True:
        # entrpoy w. noise, trainable var
        model.add(MILayer(HIDDEN_DIM, alpha=.1, trainablevar=True, initlogvar=10., add_noise=True, entropy_only=True))
    elif False:
        # noise, fixed var, no penalty
        model.add(MILayer(HIDDEN_DIM, alpha=0, trainablevar=False, initlogvar=0.1, add_noise=True, entropy_only=False))
    else:
        # noise, fixed var, entropy cost
        model.add(MILayer(HIDDEN_DIM, alpha=0.2, trainablevar=False, initlogvar=0.1, add_noise=True, entropy_only=True))

    #model.add(MILayer(HIDDEN_DIM, alpha=0.1, add_noise=True))
    #model.add(MILayer(HIDDEN_DIM, alpha=0.1, trainablevar=False, initlogvar=-.7, add_noise=True, entropy_only=False))
    #model.add(MILayer(HIDDEN_DIM, alpha=0.0, add_noise=True, entropy_only=False))


    #model.add(Dense(HIDDEN_DIM, input_dim=X_train.shape[1], activation='tanh', activity_regularizer=MIRegularizer(0, gaussian_var)))


    #model.add(Dense(HIDDEN_DIM, input_dim=HIDDEN_DIM, activation='tanh')) # , activity_regularizer=MIRegularizer(0, gaussian_var)))
    #model.add(GaussianNoise(np.sqrt(gaussian_var)))
    #model.add(Dense(HIDDEN_DIM, input_dim=HIDDEN_DIM, activation='tanh')) # , activity_regularizer=MIRegularizer(0, gaussian_var)))
    #model.add(Dense(HIDDEN_DIM, input_dim=HIDDEN_DIM, activation='tanh')) # , activity_regularizer=MIRegularizer(0, gaussian_var)))
    model.add(Dense(nb_classes, activation='softmax')) # , activity_regularizer=MIRegularizer(0, gaussian_var)))

    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

In [ ]:
if False:
    for iterndx in range(25):
        print iterndx, " ",
        #model.fit(X_train, Y_train, nb_epoch=1, batch_size=250, validation_split=0.1) # , verbose=1)
        model.fit(X_train, Y_train, nb_epoch=1, batch_size=500, validation_split=0.1) # , verbose=1)
        if hasattr(model.layers[-2],'logvar'):
            print K.get_value(model.layers[-2].logvar)


In [ ]:
activity = get_activations(model, -2, d.X_train)[0]
plot_activity(activity, colors=d.y_train) # , doPCA=False)
plt.figure()
plt.scatter(activity[:,0],activity[:,1])
plt.xlim([-1.5,1.5])
plt.ylim([-1.5,1.5])
plt.figure()
plt.plot(np.var(activity, axis=0))
plt.ylim([0, plt.ylim()[1]])


In [ ]:
#adict = {}
#adict['noreg'] = get_activations(model, 0, X_train)[0]
#adict['entropyonly'] = get_activations(model, 0, X_train)[0]
#adict['mionly'] = get_activations(model, 0, X_train)[0]
#adict['entropynnoise'] = get_activations(model, 0, X_train)[0]
#adict['minnoise'] = get_activations(model, 0, X_train)[0]
#adict['fixednoise'] = get_activations(model, 0, X_train)[0]
#adict['mifixednoise'] = get_activations(model, 0, X_train)[0]


In [ ]:
for k,v in adict.iteritems():
    plot_activity(v)
    plt.title(k)

In [ ]:
modelobj = model.model
trainweights = [kdelayer.logvar, noiselayer.logvar,]
#inputlayer = Input(tensor=X_train_Kvar)
loss =  kde_entropy(kdelayer.input, K.exp(kdelayer.logvar)+K.exp(noiselayer.logvar))
loss -= kde_condentropy(kdelayer.input, K.exp(noiselayer.logvar))

K.set_learning_phase(1)
print type(K.learning_phase())
if (modelobj.uses_learning_phase and type(K.learning_phase()) is not int):
    print 'here'
    inputs = modelobj.inputs + modelobj.targets + modelobj.sample_weights + [K.learning_phase()]
else:
    inputs = modelobj.inputs + modelobj.targets + modelobj.sample_weights
    
print modelobj.total_loss

training_updates = modelobj.optimizer.get_updates(trainweights, modelobj.constraints, modelobj.total_loss + loss)
updates = modelobj.updates + training_updates

# returns loss and metrics. Updates weights at each call.
f= K.function(inputs, [loss,], updates=training_updates)
f([d.X_train,])

In [ ]:
print modelobj.loss_functions

In [ ]:
modelobj = model.model
trainweights = [kdelayer.logvar,]
Ktrn = K.variable(d.X_train)
logvar = K.variable(0.0)
loss = kde_entropy(Ktrn, K.exp(logvar))
#mi = kde_entropy(kdelayer.input, K.exp(kdelayer.logvar)+K.exp(noiselayer.logvar)) - \
#     kde_condentropy(kdelayer.input, K.exp(noiselayer.logvar))
#loss = modelobj.total_loss + alpha * mi

#inputs = modelobj.inputs + modelobj.targets + modelobj.sample_weights + [K.learning_phase()]
updates = modelobj.optimizer.get_updates([logvar,], modelobj.constraints, [loss,])

f= K.function([logvar,], [loss,], updates=updates)
f([float(0.1)])

In [ ]:
loss = kde_entropy(Ktrn, K.exp(kdelayer.logvar))
gradval = K.gradients(loss, kdelayer.logvar)
gradf= K.function([kdelayer.logvar,], gradval)

In [ ]:
gradf([.1])

In [ ]:
import scipy.optimize
scipy.optimize.minimize(lambda x: f([d.X_train]), -10, jac=lambda x: gradf([d.X_train]), options={'disp':True}) # 'BFGS')

In [ ]:
from scipy.misc import logsumexp
def get_dists(output):
    N, dims = output.shape
            
    # Kernel density estimation of entropy
    y1 = output[None,:,:]
    y2 = output[:,None,:]
    
    dists = np.sum((y1-y2)**2, axis=2) 
    return dists

dists = get_dists(x1)

def f3(dists, N, dims, var):
    dists2 = dists / (2*var)
    
    normconst = (dims/2.0)*np.log(2*np.pi*var)

    if False:
        # Removes effect of diagonals, i.e. leave-one-out entropy
        normCount = N-1
        diagvals = 10e20*np.eye(N)
        dists2 = dists2 + diagvals
    else:
        normCount = N
    
    lprobs = logsumexp(-dists2) - np.log(normCount) - normconst
    
    h = -np.mean(lprobs)
    return h

l=[]
logvars = np.linspace(-10, 10, 20)
for logvar in logvars:
    val =  f3(dists, x1.shape[0], x1.shape[1], np.exp(logvar))
    print logvar, val
    l.append( val )
    

In [ ]:
r=    scipy.optimize.minimize(negloglikelihood, -10)


In [ ]:

def make_kdevar_update_func(model, kdelayer):
    modelobj = model.model
    trainweights = [kdelayer.logvar,]
    loss = kde_entropy(kdelayer.input, K.exp(kdelayer.logvar))
    #mi = kde_entropy(kdelayer.input, K.exp(kdelayer.logvar)+K.exp(noiselayer.logvar)) - \
    #     kde_condentropy(kdelayer.input, K.exp(noiselayer.logvar))
    #loss = modelobj.total_loss + alpha * mi

    #inputs = modelobj.inputs + modelobj.targets + modelobj.sample_weights + [K.learning_phase()]
    updates = modelobj.optimizer.get_updates(trainweights, modelobj.constraints, [loss,])
    
    return K.function([model.layers[0].input,K.learning_phase()], [loss,], updates=updates)

f=make_kdevar_update_func(model, kdelayer)

In [ ]:
nlayerinput = lambda x: K.function([model.layers[0].input, K.learning_phase()], [noiselayer.input])([x,1])[0]
vals = nlayerinput(d.X_train)
def get_dists(output):
    N, dims = output.shape

    # Kernel density estimation of entropy
    y1 = output[None,:,:]
    y2 = output[:,None,:]

    dists = np.sum((y1-y2)**2, axis=2) 
    return dists
dists = get_dists(vals)

In [ ]:
from miregularizer import kde_entropy_from_dists_loo

#dists2 = K.placeholder()
logvar2 = K.placeholder()
f = K.function([logvar2, K.learning_phase()], [kde_entropy_from_dists_loo2(dists, vals.shape[0], vals.shape[1], K.exp(logvar2))])
f2 = K.function([logvar2, K.learning_phase()], K.gradients(kde_entropy_from_dists_loo2(dists, vals.shape[0], vals.shape[1], K.exp(logvar2)), logvar2))
def negloglikelihood(logvar):
    #return f([d.X_train, 1])[0] #self.
    val =f([logvar, 1])[0]
    print logvar, val
    return  val
def jac(logvar):
    #return f([d.X_train, 1])[0] #self.
    val =f2([logvar, 1])[0]
    print 'jac', logvar, val
    return  val
print f([ 1, 1])[0]
print f2([1.1, 1])[0]
#adsf

stime = time.time()

r= scipy.optimize.minimize(lambda x: f([x, 1])[0], -10, jac=lambda x: f2([x, 1])[0])
print r.x
print time.time() - stime
#adsf

#f([d.X_train, 1])

In [ ]:
def kde_entropy_from_dists_loo_local(dists, N, dims, var):
    dists2 = dists / (2*var)

    normconst = (dims/2.0)*np.log(2*np.pi*var)

    # Removes effect of diagonals, i.e. leave-one-out entropy
    lprobs = logsumexp(-dists2, axis=1) - np.log(N-1) - normconst
    h = -np.mean(lprobs)
    return h
#print 
#print dists2.shape

def f(x):
    val = kde_entropy_from_dists_loo(dists2, vals.shape[0], vals.shape[1], np.exp(x))
    print x, val
    return val
stime = time.time()
r= scipy.optimize.minimize(lambda x: kde_entropy_from_dists_loo_local(dists2, vals.shape[0], vals.shape[1], np.exp(x)), -10, method='powell')
print time.time() - stime
print r.x
print 